In [1]:
using PlotlyJS
# using XLSX
include("../model/utils.jl")
include("../model/unit_commitment.jl")
include("../model/economic_dispatch.jl")
include("./plotting.jl")
include("./processing.jl")
# XLSX.writetable(joinpath(folder_path,"LOL_CUR.xlsx"), "LOL_CUR" => change_type(LOL_CUR, Symbol, string))

WebIO._IJuliaInit()

calculate_reserve_gcd_KPI (generic function with 1 method)

In [31]:
folder_path = joinpath("..","output", "solutions_v4.2")
solution_folders = ["n_7", "n_228", "n_259", "n_289"]
# solution_folders = ["n_7"]

keys = [:demand, :generation, :storage, :reserve, :energy_reserve, :scalar]
s_uc = [parquet_to_solution("s_uc", joinpath(folder_path, s)) for s in solution_folders]
s_ed = [parquet_to_solution("s_ed", joinpath(folder_path, s)) for s in solution_folders]
s_uc = NamedTuple(k => vcat([s[k] for s in s_uc if haskey(s, k)]...) for k in keys)
s_ed = NamedTuple(k => vcat([s[k] for s in s_ed if haskey(s, k)]...) for k in keys)
;

reading...
../output/solutions_v4.2/n_7/s_uc_demand.parquet
../output/solutions_v4.2/n_7/s_uc_generation.parquet
../output/solutions_v4.2/n_7/s_uc_storage.parquet
../output/solutions_v4.2/n_7/s_uc_reserve.parquet
../output/solutions_v4.2/n_7/s_uc_energy_reserve.parquet
../output/solutions_v4.2/n_7/s_uc_scalar.parquet
...done
reading...
../output/solutions_v4.2/n_228/s_uc_demand.parquet
../output/solutions_v4.2/n_228/s_uc_generation.parquet
../output/solutions_v4.2/n_228/s_uc_storage.parquet
../output/solutions_v4.2/n_228/s_uc_reserve.parquet
../output/solutions_v4.2/n_228/s_uc_energy_reserve.parquet
../output/solutions_v4.2/n_228/s_uc_scalar.parquet
...done
reading...
...done
reading...
...done
reading...
../output/solutions_v4.2/n_7/s_ed_demand.parquet
../output/solutions_v4.2/n_7/s_ed_generation.parquet
../output/solutions_v4.2/n_7/s_ed_storage.parquet
../output/solutions_v4.2/n_7/s_ed_reserve.parquet
../output/solutions_v4.2/n_7/s_ed_energy_reserve.parquet
../output/solutions_v4.2/n

In [32]:
gcdi_KPI_adequacy = calculate_adecuacy_gcdi_KPI(s_ed, s_uc)
gcd_KPI_adequacy = calculate_adecuacy_gcd_KPI(gcdi_KPI_adequacy)
KPI_reserve = calculate_reserve_KPI(s_ed, s_uc)
gcdi_KPI_reserve = calculate_reserve_gcdi_KPI(KPI_reserve)
gcd_KPI_reserve = calculate_reserve_gcd_KPI(gcdi_KPI_reserve)
# s_uc_scalar = include_Δobjective_value(s_uc.scalar, [:day])
# transform!(s_uc_scalar, :configuration .=> ByRow(x -> parse_configuration_to_mu(x)) .=> :mu)
# sort!(s_uc_scalar, :mu)
;

In [33]:
# plot(gcdi_KPI_adequacy, x = :CUR_MWh, y = :LGEN_MWh, color = :mu, text = :iteration, mode = "markers")


In [34]:
unique(s_ed.generation.configuration)

5-element Vector{Symbol}:
 :base_ramp_storage_energy_reserve_cumulated
 :base_ramp_storage_envelopes_up_1_dn_1
 :base_ramp_storage_envelopes_up_0_5_dn_0_5
 :base_ramp_storage_envelopes_up_0_dn_0
 :base_ramp_storage_envelopes_up_0_25_dn_0_25

In [35]:
# function check_loq_value(df, field, value, accuracy)
#     filter = (df[!,field] .> value) .& (df[!,field] .< Inf) .& (.!isnan.(df[!,field]))
#     filter = (df[!,field].*filter .-value) .< accuracy
#     println(all(filter))
#     return df[.!filter,:]

# end
# # Check
# accuracy = 0.000001
# check_loq_value(KPI_reserve, :delivered_r_up_ratio, 1, accuracy)
# check_loq_value(KPI_reserve, :delivered_r_dn_ratio, 1, accuracy)
;
# balance = (KPI_reserve.required_r_up_MW .-KPI_reserve.required_r_dn_MW) .- (KPI_reserve.delivered_r_up_MW .- KPI_reserve.delivered_r_dn_MW .+ KPI_reserve.LOL_MW.*(required.>=0))
# println(all(abs.(balance) .<= accuracy))
# Todo: re-calcualte balance balance


In [36]:
# to_plot = gcdi_KPI_adequacy[abs.(gcdi_KPI_adequacy.Δobjective_value_relative_ref_conf),:]
p1 = plot(gcdi_KPI_adequacy, y = :objective_value, x = :ENS_MWh, mode = "markers", facet_col = :day, color= :configuration, text =:iteration)
to_plot = gcdi_KPI_adequacy[abs.(gcdi_KPI_adequacy.Δobjective_value_relative_ref_conf) .>0.001,:]
p2 = plot(gcdi_KPI_adequacy, y = :Δobjective_value_relative_ref_conf, x = :ENS_MWh, mode = "markers", facet_col = :day, color= :configuration, text =:iteration)
relayout([p1;p2], height=800)

data: [
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields height, margin, template, xaxis1, xaxis2, yaxis1, and yaxis2"

In [37]:
p1 = plot(gcd_KPI_adequacy, x = :mu, y = :EOV, color = :day)
p2 = plot(gcd_KPI_adequacy, x = :mu, y = :EENS, color = :day)
p3 = plot(gcd_KPI_adequacy, x = :mu, y = :ECUR, color = :day)
# p4 = plot(gcd_KPI_adequacy, x = :mu, y = :ELGEN, color = :day)
# p3 = plot(KPI, x = :mu, y = :LOLE, color = :day)
[p2 p3]

data: [
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields margin, template, xaxis1, xaxis2, yaxis1, and yaxis2"

In [38]:
p1 = plot(gcd_KPI_reserve, x = :mu, y = :delivered_r_up_ratio, color = :day)
p2 = plot(gcd_KPI_reserve, x = :mu, y = :delivered_r_dn_ratio, color = :day)
p3 = plot(gcd_KPI_reserve, x = :mu, y = :E_delivered_r_up_MWh, color = :day)
p4 = plot(gcd_KPI_reserve, x = :mu, y = :E_deliverered_dn_MWh, color = :day)
# [p1 p2; p3 p4]
[p1 p2]

data: [
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields margin, template, xaxis1, xaxis2, yaxis1, and yaxis2"

In [39]:
p1 = plot(gcdi_KPI_adequacy, y = :Δobjective_value_relative_ref_conf, x = :mu, facet_col = :day, color = :iteration)
p2 = plot(gcdi_KPI_adequacy, y = :ENS_MWh, x = :mu,  facet_col = :day,color = :iteration)
p3 = plot(gcdi_KPI_adequacy, y = :LGEN_MWh, x = :mu, facet_col = :day,color = :iteration)
p4 = plot(gcdi_KPI_adequacy, y = :CUR_MWh, x = :mu,  facet_col = :day,color = :iteration)

[p1 p2; p3 p4]

data: [
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and y

In [40]:
# plot(LOL_CUR, x = :LOLE, y = :EENS, facet_col = :day, group = :configuration, marker_color = :EOV_M, mode = "markers", marker=attr(showscale=true))
# configs
# plot(gcd_KPI_adequacy, x = :ECUR, y = :EENS, facet_col = :day, mode = "markers", text = :day, color=:configuration) # marker=attr(showscale=true, coloraxis="coloraxis", color=:day

In [41]:
# plot(LOL_CUR, x = :EOV, y = :EENS, text = :configuration, color = :configuration, mode = "markers")
# plot(gcd_KPI_adequacy, x = :EOV, y = :EENS, color = :configuration, mode = "markers", facet_col = :day)


In [42]:
group_by = [:configuration, :day]
supply_uc, demand_uc = calculate_supply_demand(s_uc, union([:hour, :resource], group_by))
supply_ed, demand_ed = calculate_supply_demand(s_ed, union([:hour, :resource, :iteration], group_by))
reserve_uc = calculate_reserve(s_uc.reserve, nothing, union([:hour, :resource], group_by))
reserve_ed = calculate_reserve(s_ed.reserve, nothing, union([:hour, :resource, :iteration], group_by))

commit_uc = combine(groupby(s_uc.generation, [:resource, :configuration, :day, :hour]), :commit => sum => :commit)
commit_ed = combine(groupby(s_ed.generation, [:resource, :configuration, :day, :iteration, :hour]), :commit => sum => :commit)
;

In [43]:
day_ = 7
iteration_ = :demand_40

:demand_40

### Checking reserve delivey

In [44]:
config_ = :base_ramp_storage_envelopes_up_0_3_dn_0_3
KPI_reserve_ = KPI_reserve[(KPI_reserve.configuration .== config_) .& (KPI_reserve.day .== day_) .& (KPI_reserve.iteration .== iteration_), :]
p = [scatter(KPI_reserve_, x=:hour, y=:required_r_up_MW, name = "required_r_up_MW"),scatter(KPI_reserve_, x=:hour, y=:delivered_r_up_MW, name = "delivered_r_up_MW")]
p1 = plot(p, Layout(yaxis_title="reserve up MW", xaxis_title="hour"))
p = [scatter(KPI_reserve_, x=:hour, y=:required_r_dn_MW, name = "required_r_dn_MW"),scatter(KPI_reserve_, x=:hour, y=:delivered_r_dn_MW, name = "delivered_r_dn_MW")]
p2 = plot(p, Layout(yaxis_title="reserve dn MW", xaxis_title="hour"))
[p1 p2]

data: [
  "scatter with fields name, type, x, xaxis, y, and yaxis",
  "scatter with fields name, type, x, xaxis, y, and yaxis",
  "scatter with fields name, type, x, xaxis, y, and yaxis",
  "scatter with fields name, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields margin, template, xaxis1, xaxis2, yaxis1, and yaxis2"

In [45]:
group_by_big= [:configuration, :day]
reserve_ = combine(groupby(s_uc.reserve, union(group_by_big, [:resource])), [:reserve_up_MW,:reserve_down_MW]  .=> sum .=> [:reserve_up_MW,:reserve_down_MW])
combine(groupby(reserve_, union(group_by_big, [])), [:reserve_up_MW,:reserve_down_MW]  .=> sum .=> [:reserve_up_total_MW,:reserve_down_total_MW])
leftjoin!(reserve_,
combine(groupby(reserve_, union(group_by_big, [])), [:reserve_up_MW,:reserve_down_MW]  .=> sum .=> [:reserve_up_total_MW,:reserve_down_total_MW]),
on = group_by_big
)

reserve_.reserve_down_relative = reserve_.reserve_down_MW ./ reserve_.reserve_down_total_MW
reserve_.reserve_up_relative = reserve_.reserve_up_MW ./ reserve_.reserve_up_total_MW
transform!(reserve_, :configuration .=> ByRow(x -> parse_configuration_to_mu(x)) .=> :mu)
sort!(reserve_, :mu)
;

In [46]:
fieldy = :resource
fieldx = :reserve_up_relative
df = reserve_[reserve_.day .== 7,:] 
plot([scatter(
          x= df[df[!,fieldy] .== r, :mu], y=df[df[!,fieldy] .== r, fieldx],
          stackgroup=map_stack_group(r,"required"), mode="lines", name = r,
          line=attr(width=1, color=color_discrete_map(r), shape = "line")
      ) for r in unique(df[!,fieldy])],
    Layout(yaxis_title=string(fieldx), xaxis_title=string(:mu), title = string(fieldx))
)


data: [
  "scatter with fields line, mode, name, stackgroup, type, x, and y",
  "scatter with fields line, mode, name, stackgroup, type, x, and y",
  "scatter with fields line, mode, name, stackgroup, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

### Checking dispatch

In [47]:
day_ = 7
iteration_ = :demand_21

:demand_21

In [48]:
# Checking solutions are different
scalar = s_ed.scalar[(s_ed.scalar.day .== day_) .& (s_ed.scalar.iteration .== iteration_),:]
(scalar[scalar.configuration .== :base_ramp_storage_envelopes_up_1_dn_1, :objective_value] - scalar[scalar.configuration .== :base_ramp_storage_envelopes_up_0_dn_0, :objective_value]) / scalar[scalar.configuration .== :base_ramp_storage_envelopes_up_0_dn_0, :objective_value]

1×1 Matrix{Float64}:
 1.985705814752361

In [49]:
scalar[scalar.configuration .== :base_ramp_storage_envelopes_up_0_4_dn_0_4, :objective_value]

Float64[]

In [50]:
config_ = :base_ramp_storage_envelopes_up_0_dn_0
supply_uc_ = supply_uc[(supply_uc.configuration .== config_) .& (supply_uc.day .== day_), :]
demand_uc_ = demand_uc[(demand_uc.configuration .== config_) .& (demand_uc.day .== day_), :]
reserve_uc_ = reserve_uc[(reserve_uc.configuration .== config_) .& (reserve_uc.day .== day_), :]
plot_supply_demand(supply_uc_, demand_uc_, string(config_))

data: [
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, margin, template, title, xaxis1, xaxis2, yaxis1, and yaxis2"

In [51]:
plot_reserve(reserve_uc_)

data: [
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, margin, template, xaxis1, xaxis2, yaxis1, and yaxis2"

In [52]:
config_ = :base_ramp_storage_envelopes_up_0_95_dn_0_95
supply_uc_ = supply_uc[(supply_uc.configuration .== config_) .& (supply_uc.day .== day_), :]
demand_uc_ = demand_uc[(demand_uc.configuration .== config_) .& (demand_uc.day .== day_), :]
reserve_uc_ = reserve_uc[(reserve_uc.configuration .== config_) .& (reserve_uc.day .== day_), :]
plot_supply_demand(supply_uc_, demand_uc_, string(config_))

data: []

layout: "layout with fields annotations, margin, template, title, xaxis1, xaxis2, yaxis1, and yaxis2"

In [53]:
plot_reserve(reserve_uc_)

data: []

layout: "layout with fields annotations, margin, template, xaxis1, xaxis2, yaxis1, and yaxis2"

In [54]:
config_ = :base_ramp_storage_envelopes_up_0_dn_0
supply_ed_ = supply_ed[(supply_ed.configuration .== config_) .& (supply_ed.day .== day_) .& (supply_ed.iteration .== iteration_), :]
demand_ed_ = demand_ed[(demand_ed.configuration .== config_) .& (demand_ed.day .== day_) .& (demand_ed.iteration .== iteration_), :]
reserve_ed_ = reserve_ed[(reserve_ed.configuration .== config_) .& (reserve_ed.day .== day_), :]
plot_supply_demand(supply_ed_, demand_ed_, string(config_))


data: [
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, margin, template, title, xaxis1, xaxis2, yaxis1, and yaxis2"

In [55]:
plot_reserve(reserve_ed_)

data: [
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, margin, template, xaxis1, xaxis2, yaxis1, and yaxis2"

In [56]:
to_plot = commit_ed[(commit_ed.configuration .== config_) .&  (commit_ed.day .== day_) .&  (commit_ed.iteration .== iteration_),:] 
p1 = plot(to_plot, x = :hour, y = :commit, color = :resource, kind = "bar")

to_plot = s_ed.storage[(s_ed.storage.configuration .== config_) .&  (s_ed.storage.day .== day_) .&  (s_ed.storage.iteration .== iteration_),:]
p2 = plot(to_plot, x = :hour , y = :SOE_MWh, color = :resource, kind = "bar", Layout(barmode="stack")) 
[p1 p2]

data: [
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields margin, template, xaxis1, xaxis2, yaxis1, and yaxis2"

In [57]:
config_ = :base_ramp_storage_envelopes_up_0_4_dn_0_4
supply_ed_ = supply_ed[(supply_ed.configuration .== config_) .& (supply_ed.day .== day_) .& (supply_ed.iteration .== iteration_), :]
demand_ed_ = demand_ed[(demand_ed.configuration .== config_) .& (demand_ed.day .== day_) .& (demand_ed.iteration .== iteration_), :]
reserve_ed_ = reserve_ed[(reserve_ed.configuration .== config_) .& (reserve_ed.day .== day_), :]
plot_supply_demand(supply_ed_, demand_ed_, string(config_))


data: []

layout: "layout with fields annotations, margin, template, title, xaxis1, xaxis2, yaxis1, and yaxis2"

In [58]:
plot_reserve(reserve_ed_)

data: []

layout: "layout with fields annotations, margin, template, xaxis1, xaxis2, yaxis1, and yaxis2"

In [59]:
to_plot = commit_ed[(commit_ed.configuration .== config_) .&  (commit_ed.day .== day_) .&  (commit_ed.iteration .== iteration_),:] 
p1 = plot(to_plot, x = :hour, y = :commit, color = :resource, kind = "bar")

to_plot = s_ed.storage[(s_ed.storage.configuration .== config_) .&  (s_ed.storage.day .== day_) .&  (s_ed.storage.iteration .== iteration_),:]
p2 = plot(to_plot, x = :hour , y = :SOE_MWh, color = :resource, kind = "bar", Layout(barmode="stack")) 
[p1 p2]


LoadError: obsvals is empty, but order_keys is not

### Studying oscilations

In [ ]:
day_ = 7
iteration_ = :demand_10

In [ ]:
to_plot = commit_ed[(commit_ed.configuration .==  :base_ramp_storage_envelopes_up_0_dn_0) .&  (commit_ed.day .== day_) .&  (commit_ed.iteration .== iteration_),:] 
p1 = plot(to_plot, x = :hour, y = :commit, color = :resource, kind = "bar", title = "mu = 0.6")
to_plot = commit_ed[(commit_ed.configuration .==  :base_ramp_storage_envelopes_up_0_4_dn_0_4) .&  (commit_ed.day .== day_) .&  (commit_ed.iteration .== iteration_),:] 
p2 = plot(to_plot, x = :hour, y = :commit, color = :resource, kind = "bar" , title = "mu = 0.6")
[p1 p2]

In [ ]:
config_ = :base_ramp_storage_envelopes_up_0_dn_0
supply_uc_ = supply_uc[(supply_uc.configuration .== config_) .& (supply_uc.day .== day_), :]
demand_uc_ = demand_uc[(demand_uc.configuration .== config_) .& (demand_uc.day .== day_), :]
plot_supply_demand(supply_uc_, demand_uc_, string(config_))

In [ ]:
to_plot = commit_ed[(commit_ed.configuration .== config_) .&  (commit_ed.day .== day_) .&  (commit_ed.iteration .== iteration_),:] 
p1 = plot(to_plot, x = :hour, y = :commit, color = :resource, kind = "bar")

to_plot = s_ed.storage[(s_ed.storage.configuration .== config_) .&  (s_ed.storage.day .== day_) .&  (s_ed.storage.iteration .== iteration_),:]
p2 = plot(to_plot, x = :hour , y = :SOE_MWh, color = :resource, kind = "bar", Layout(barmode="stack")) 
[p1 p2]

In [ ]:
config_ = :base_ramp_storage_envelopes_up_0_6_dn_0_6
supply_uc_ = supply_uc[(supply_uc.configuration .== config_) .& (supply_uc.day .== day_), :]
demand_uc_ = demand_uc[(demand_uc.configuration .== config_) .& (demand_uc.day .== day_), :]
plot_supply_demand(supply_uc_, demand_uc_, string(config_))

In [ ]:
to_plot = commit_ed[(commit_ed.configuration .== config_) .&  (commit_ed.day .== day_) .&  (commit_ed.iteration .== iteration_),:] 
p1 = plot(to_plot, x = :hour, y = :commit, color = :resource, kind = "bar")

to_plot = s_ed.storage[(s_ed.storage.configuration .== config_) .&  (s_ed.storage.day .== day_) .&  (s_ed.storage.iteration .== iteration_),:]
p2 = plot(to_plot, x = :hour , y = :SOE_MWh, color = :resource, kind = "bar", Layout(barmode="stack")) 
[p1 p2]

In [ ]:
KPI_adecuacy